In [ ]:
from google.colab import drive
import os

# 1. Google Driveをマウント
drive.mount('/content/drive')

# 2. プロジェクトベースディレクトリを指定
base_dir = '/content/drive/MyDrive/Colab Notebooks/NFFA-SEM'

# 3. 作りたいサブディレクトリリスト
dirs = [
    'data/original',
    'data/processed',
    'models',
    'scripts',
    'checkpoints',
    'logs',
    'configs',
    'utils'
]

# 4. ディレクトリを一括作成
for d in dirs:
    path = os.path.join(base_dir, d)
    os.makedirs(path, exist_ok=True)

# 5. README.mdも作っておく（空ファイル）
readme_path = os.path.join(base_dir, 'README.md')
with open(readme_path, 'w') as f:
    f.write('# NFFA-SEM\n\nProject description here.')


Mounted at /content/drive


In [ ]:
# 6. .gitignore作成
gitignore_path = os.path.join(base_dir, '.gitignore')
gitignore_content = """
# Python関連
__pycache__/
*.py[cod]
*.so

# 仮想環境
venv/
env/
ENV/
*.egg-info/

# Jupyter Notebookチェックポイント
.ipynb_checkpoints/

# データ・チェックポイント
data/original/
data/processed/
checkpoints/
logs/

# モデル保存ファイル
*.pth
*.pt
*.h5

# 設定ファイルの一時保存ファイル
*.yaml~
*.json~

# IDE関連（VSCode, PyCharmなど）
.vscode/
.idea/

# Mac特有の隠しファイル
.DS_Store

# Google Colab特有
/content/

# その他一時ファイル
*.log
*.tmp
*.bak
"""

with open(gitignore_path, 'w') as f:
    f.write(gitignore_content.strip())

print("✅ プロジェクトディレクトリと .gitignore を作成しました！")

✅ プロジェクトディレクトリと .gitignore を作成しました！


In [ ]:
# 7. scripts/train.py を作成（config.yaml参照）
train_script_path = os.path.join(base_dir, 'scripts', 'train.py')
train_script_content = """
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os
import yaml

# ------------------------------
# Configファイルの読み込み
# ------------------------------
config_path = '../configs/config.yaml'

with open(config_path) as f:
    config = yaml.safe_load(f)

batch_size = config['train']['batch_size']
epochs = config['train']['epochs']
learning_rate = config['train']['learning_rate']
model_name = config['train']['model_name']

train_dir = config['data']['train_dir']
val_dir = config['data']['val_dir']
checkpoint_dir = config['save']['checkpoint_dir']

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ------------------------------
# データセット読み込み
# ------------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# ------------------------------
# モデル準備
# ------------------------------
if model_name == 'resnet18':
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, len(train_dataset.classes))
else:
    raise ValueError(f"Unsupported model: {model_name}")

model = model.to(device)

# 損失関数と最適化手法
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ------------------------------
# 学習ループ
# ------------------------------
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# ------------------------------
# モデル保存
# ------------------------------
os.makedirs(checkpoint_dir, exist_ok=True)
save_path = os.path.join(checkpoint_dir, 'best_model.pth')
torch.save(model.state_dict(), save_path)
print(f"✅ モデル保存完了！ 保存先: {save_path}")
"""
with open(train_script_path, 'w') as f:
    f.write(train_script_content.strip())

print("✅ プロジェクトディレクトリ＋初期スクリプトを作成しました！")

✅ プロジェクトディレクトリ＋初期スクリプトを作成しました！


In [ ]:
# configs/config.yaml
config_path = os.path.join(base_dir, 'configs', 'config.yaml') # Changed 'config' to 'configs'
config_content = """
train:
  batch_size: 32
  epochs: 10
  learning_rate: 0.001
  model_name: resnet18

data:
  train_dir: ../data/processed/train
  val_dir: ../data/processed/val

save:
  checkpoint_dir: ../checkpoints
"""
with open(config_path, 'w') as f:
    f.write(config_content.strip())

print("✅ config.yamlを作成しました！")

✅ config.yamlを作成しました！


In [ ]:
# 7. scripts/train.py を作成（tensorBoard版）
train_script_path = os.path.join(base_dir, 'scripts', 'train2.py')
train_script_content = """
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os
import yaml
from torch.utils.tensorboard import SummaryWriter

# ------------------------------
# Configファイルの読み込み
# ------------------------------
config_path = '../configs/config.yaml'

with open(config_path) as f:
    config = yaml.safe_load(f)

batch_size = config['train']['batch_size']
epochs = config['train']['epochs']
learning_rate = config['train']['learning_rate']
model_name = config['train']['model_name']

train_dir = config['data']['train_dir']
val_dir = config['data']['val_dir']
checkpoint_dir = config['save']['checkpoint_dir']

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ------------------------------
# データセット読み込み
# ------------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# ------------------------------
# モデル準備
# ------------------------------
if model_name == 'resnet18':
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, len(train_dataset.classes))
else:
    raise ValueError(f"Unsupported model: {model_name}")

model = model.to(device)

# 損失関数と最適化手法
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ------------------------------
# TensorBoardライター作成
# ------------------------------
log_dir = '../logs'
os.makedirs(log_dir, exist_ok=True)
writer = SummaryWriter(log_dir=log_dir)

# ------------------------------
# 学習ループ
# ------------------------------
for epoch in range(epochs):
    # 学習モード
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = correct / total

    # ------------------------------
    # バリデーション
    # ------------------------------
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_loss /= len(val_loader)
    val_acc = correct / total

    # ------------------------------
    # ログ出力
    # ------------------------------
    print(f"Epoch [{epoch+1}/{epochs}] Train Loss: {train_loss:.4f} Val Loss: {val_loss:.4f} Train Acc: {train_acc:.4f} Val Acc: {val_acc:.4f}")
    writer.add_scalar('Loss/Train', train_loss, epoch)
    writer.add_scalar('Loss/Validation', val_loss, epoch)
    writer.add_scalar('Accuracy/Train', train_acc, epoch)
    writer.add_scalar('Accuracy/Validation', val_acc, epoch)

# ------------------------------
# モデル保存
# ------------------------------
os.makedirs(checkpoint_dir, exist_ok=True)
save_path = os.path.join(checkpoint_dir, 'best_model.pth')
torch.save(model.state_dict(), save_path)
print(f"✅ モデル保存完了！ 保存先: {save_path}")

# ------------------------------
# TensorBoardライター閉じる
# ------------------------------
writer.close()
"""
with open(train_script_path, 'w') as f:
    f.write(train_script_content.strip())

print("✅ TensorBoard版train2.pyを作成しました！")

✅ プロジェクトディレクトリ＋初期スクリプトを作成しました！
